In [2]:
import json
from collections import OrderedDict
from itertools import repeat
import numpy as np

In [3]:
# with open('./DPR/data/retriever/nq-train.json') as f:
#     data = json.load(f)
# data[0]

In [4]:
with open('./movieQA/train.json') as f:
    train_moviedata = json.load(f)
with open('./movieQA/test.json') as f:
    test_moviedata = json.load(f)
with open('./movieQA/dev.json') as f:
    dev_moviedata = json.load(f)


In [5]:
print(len(train_moviedata))
print(len(test_moviedata))
print(len(dev_moviedata))

38417
4327
4333


## add special mark near ans

In [6]:
# train


for i in range(len(train_moviedata)):
    data = train_moviedata[i]
    did = data['DID']
    question = data['QUESTIONS'][0]
    passage = data['DTEXT']
    q = question['QTEXT']
    answers = question['ANSWER'][0]['ATOKEN']
    answers.sort(key=lambda x:x['start'])


    added_count = 0
    for answer in answers:
        start = answer['start'] + added_count
        end = answer['end'] + added_count
        passage = passage[:start] + '$$' + passage[start:end] + '$$' + passage[end:]

        added_count += 4
    train_moviedata[i]['DTEXT'] = passage
    
# dev

for i in range(len(dev_moviedata)):
    data = dev_moviedata[i]
    did = data['DID']
    question = data['QUESTIONS'][0]
    passage = data['DTEXT']
    q = question['QTEXT']
    answers = question['ANSWER'][0]['ATOKEN']
    answers.sort(key=lambda x:x['start'])


    added_count = 0
    for answer in answers:
        start = answer['start'] + added_count
        end = answer['end'] + added_count
        passage = passage[:start] + '$$' + passage[start:end] + '$$' + passage[end:]

        added_count += 4
    dev_moviedata[i]['DTEXT'] = passage
    
    
# test


for i in range(len(test_moviedata)):
    data = test_moviedata[i]
    did = data['DID']
    question = data['QUESTIONS'][0]
    passage = data['DTEXT']
    q = question['QTEXT']
    answers = question['ANSWER'][0]['ATOKEN']
    answers.sort(key=lambda x:x['start'])


    added_count = 0
    for answer in answers:
        start = answer['start'] + added_count
        end = answer['end'] + added_count
        passage = passage[:start] + '$$' + passage[start:end] + '$$' + passage[end:]

        added_count += 4
    test_moviedata[i]['DTEXT'] = passage


In [7]:
movie_passages = {}

In [8]:
# split data
chunk_len = 0
chunk_count = 0
for movie_data in [train_moviedata,test_moviedata,dev_moviedata]:
    for i in range(len(movie_data)):
        data = movie_data[i]
        passage = data['DTEXT']
        did = data['DID']
        movie_passages[did] = []
        
        passage = passage.replace('!','.').replace('?','.')
        if passage.find('.') == -1:
            passage = passage + '.'
        
        chunks = []
        pos = 0
        last_pos = 0
        text = ''
        word_count = 0
        while(True):
            pos = passage.find('.',pos)
            if pos == -1:
                break
            pos += 1
            chunk = passage[last_pos:pos]
            chunk = chunk.replace('\n','')
            chunk = chunk.strip()
            word_count += len(chunk.split())
            if word_count >= 100:
#                 text = text.replace('$$','')
                chunks.append(text)
                text = chunk
                word_count = len(chunk.split())
            else:
                text += chunk
            last_pos = pos
        if text != '':
#             text = text.replace('$$','')
            chunks.append(text)
            
        movie_passages[did] = chunks


In [9]:
all_chunks = []

# when training it should only use train and dev

for movie_data in [train_moviedata,dev_moviedata,test_moviedata]:
    for i in range(len(movie_data)):
        data = movie_data[i]
        did = data['DID']
        chunks = movie_passages[did]
        for chunk in chunks:
            if len(chunk) < 10:
                continue
            all_chunks.append(chunk.replace('$$',''))
            
print(len(all_chunks))
all_chunks = list(set(all_chunks))
print(len(all_chunks))


133488
86360


In [10]:
count = 0
for chunk in all_chunks:
    count += len(chunk.split())
    
print(count/len(all_chunks))

72.06725335803613


In [11]:
#  for cal bm25
movie_question2passages = {}

In [12]:
import string
def process_chunk(chunk):
    chunk = chunk.lower()
    chunk = chunk.translate(str.maketrans('', '', string.punctuation))
    return chunk
    

In [13]:
cant_find = 0
train_instances = []
movie_data = test_moviedata
# movie_data = dev_moviedata
# movie_data = train_moviedata



for i in range(len(movie_data)):
    train_instance = {}
    data = movie_data[i]
    passage = data['DTEXT']
    did = data['DID']
    question = data['QUESTIONS'][0]

    q = question['QTEXT']
    train_instance['question'] = q
    answers = question['ANSWER'][0]['ATOKEN']
    train_instance['answers'] = []
    train_instance['positive_ctxs'] = []
    chunks = movie_passages[did]
    
    passages_label = []
    
    for ans in answers:
        padded_text = '$$' + ans['text'] + '$$'
        find_flag = False
        for chunk in chunks:
            if padded_text in chunk:
                postive_ctx = {}
                chunk = chunk.replace('$$','')
#                 chunk = process_chunk(chunk)
                postive_ctx['text'] = chunk
                postive_ctx['title'] = ''
                train_instance['answers'].append(ans['text'])
                train_instance['positive_ctxs'].append(postive_ctx)
                find_flag = True
                passages_label.append(postive_ctx['text'])
                break
        if not find_flag:
            cant_find += 1
    train_instances.append(train_instance)
    
    movie_question2passages[question['QTEXT']] = passages_label
    


print(cant_find)

188


In [14]:
train_instances[0]

{'question': 'what does Grégoire Colin appear in?',
 'answers': ['Before the Rain'],
 'positive_ctxs': [{'text': 'Grégoire Colin (born 27 July 1975) is a French actor, producer, screenwriter, and director.Before the Rain (Macedonian: Пред дождот, Pred doždot) is a 1994 film written and directed by Milcho Manchevski.The film stars Katrin Cartlidge, Rade Šerbedžija, Grégoire Colin and Labina Mitevska.It features an original score by the Macedonian band Anastasia.Before the Rain consists of three interlocking stories set both in the Republic of Macedonia and London.The film addresses the ethnic and religious roots of violence and hatred in the Balkans as well as some international implications of the tragic regional conflict.',
   'title': ''}]}

## HARD NEGATIVE SAMPLES

In [66]:
# BM25 preprocessing
from nltk.stem.porter import PorterStemmer
import string
import re
from gensim.summarization import bm25

p_stemmer = PorterStemmer()  

processed_all_chunks = []
for chunk in all_chunks:
    chunk = process_chunk(chunk)

    chunk_split = chunk.split()
    stemmed_tokens = [p_stemmer.stem(i) for i in chunk_split]
    processed_all_chunks.append(stemmed_tokens)
    
bm25_obj = bm25.BM25(processed_all_chunks)

In [77]:
idx = 0
print(len(train_instances))
get_postive_ans = 0
for train_instance in train_instances:
    question = train_instance['question']
    train_instance['negative_ctxs'] = []
    train_instance['hard_negative_ctxs'] = []
    postive_passages = [x['text'] for x in train_instance['positive_ctxs']]


    question = process_chunk(question)
    question_split = question.split()
    question_stemmed_tokens = [p_stemmer.stem(i) for i in question_split]
    
    scores = bm25_obj.get_scores(question_stemmed_tokens)
    topk = 30
    scores = np.array(scores)
    best_docs = scores.argsort()[-topk:][::-1]
    count = 0
    

    for doc_idx in best_docs:
        chunk = all_chunks[doc_idx]
        
        if chunk not in postive_passages:
            negative_ctx = {}
            negative_ctx['title'] = ''
            negative_ctx['text'] = chunk
            count += 1
            train_instance['hard_negative_ctxs'].append(negative_ctx)
        else:
            get_postive_ans += 1

        if count >= 10:
            print(f'\r{idx}, {get_postive_ans}',end='')
            idx += 1
            break
#     print(train_instance)
#     break
    

4333
4332, 3472

In [78]:
len(train_instances)

4333

In [79]:
train_instances[-1]

{'question': 'which movie did Bob Brunner write?',
 'answers': ['Exit to Eden'],
 'positive_ctxs': [{'text': 'Robert "Bob" Brunner (August 3, 1934 – October 28, 2012) was an American screenwriter, film producer, and television producer.He frequently collaborated in film and television with Garry Marshall, the creator of Happy Days.Brunner is credited with creating the "Fonzie" nickname for Henry Winkler\'s character, Arthur Fonzarelli, on Happy Days.He also created one of Fonzie\'s key catchphrases, "Sit on it."Exit to Eden is a 1994 American comedy-thriller film directed by Garry Marshall and adapted to the screen by Deborah Amelon and Bob Brunner from Anne Rampling\'s novel of the same name.',
   'title': ''}],
 'negative_ctxs': [],
 'hard_negative_ctxs': [{'title': '',
   'text': 'The Other Sister is a 1999 American romantic comedy film directed by Garry Marshall and stars Juliette Lewis, Giovanni Ribisi, Diane Keaton, and Tom Skerritt.It was filmed in Long Beach, Pasadena, and San 

## NEGATIVE SAMPLES

In [80]:
import random

for train_instance in train_instances:
    postive_passages = [x['text'] for x in train_instance['positive_ctxs']]
    hard_negative_passages = [x['text'] for x in train_instance['hard_negative_ctxs']]
    
    train_instance['negative_ctxs'] = []
    
    negative_candidate = random.choices(all_chunks , k=30)

    neg_count = 0 
    for negative_chunk in negative_candidate:
        if (negative_chunk not in postive_passages) and (negative_chunk not in hard_negative_passages):
            negative_ctx = {}
            negative_ctx['title'] = ''
            negative_ctx['text'] = negative_chunk
            neg_count += 1 
            train_instance['negative_ctxs'].append(negative_ctx)
        if neg_count >= 16:
            break
        

In [81]:
len(train_instances)

4333

In [82]:
with open('./DPR/data/retriever/movie-dev.json', 'w') as outfile:
    json.dump(train_instances, outfile)

In [288]:
print('done')

done


## check training file

In [225]:
with open('./DPR/data/retriever/movie-dev.json') as f:
    train_instances = json.load(f)
print(len(train_instances))

4333


In [230]:
idx =  -1

print(train_instances[idx]['question'])
print(train_instances[idx]['positive_ctxs'])
print('------')
print(train_instances[idx]['hard_negative_ctxs'])


which movie did Bob Brunner write?
[{'text': 'Robert "Bob" Brunner (August 3, 1934 – October 28, 2012) was an American screenwriter, film producer, and television producer.He frequently collaborated in film and television with Garry Marshall, the creator of Happy Days.Brunner is credited with creating the "Fonzie" nickname for Henry Winkler\'s character, Arthur Fonzarelli, on Happy Days.He also created one of Fonzie\'s key catchphrases, "Sit on it."Exit to Eden is a 1994 American comedy-thriller film directed by Garry Marshall and adapted to the screen by Deborah Amelon and Bob Brunner from Anne Rampling\'s novel of the same name.', 'title': ''}]
------
[{'title': '', 'text': 'Douglas was one of the most popular American authors of his time, although he did not write his first novel until he was 50.'}, {'title': '', 'text': 'It did however only win only four awards, Best Actor, Best Actress, Best Editor and Best Make-up.Her next feature film is called Dancers, and the writing has again

## bulit all tsv file for dense embedding

In [313]:
# build tsv file
# format doc_id, doc_text, title

fp = open('./DPR/data/retriever/movie-dpr.tsv','w')

for movie_data in [test_moviedata,train_moviedata,dev_moviedata]:
# for movie_data in [test_moviedata]:
    for i in range(len(movie_data)):
        count = 0
        data = movie_data[i]
        did = data['DID']
        original_passage = data['DTEXT']
#         print(original_passage)
        
        
#         print('--------------')
        chunks = movie_passages[did]
#         print(len(chunks))
        
        chunks = list(set(chunks))


        for chunk in chunks:
            doc_id = did + '_' + str(count)
            count += 1
            doc_text = chunk.replace('$$','')
            title = ''
            line = f'{doc_id}\t{doc_text}\t{title}'
            
#             print(line)

            print(line,file=fp)
fp.close()


In [344]:
import pickle

In [349]:
file = open('./DPR/movie_test_0.pkl', 'rb')
test = pickle.load(file)
file.close()


In [355]:
test[0][1].shape

(768,)

## prepare qa-file for testing

In [56]:
fp = open('./DPR/data/retriever/movie-test.tsv','w')


for i in range(len(test_moviedata)):
    data = test_moviedata[i]
    passage = data['DTEXT']
    did = data['DID']
    question = data['QUESTIONS'][0]['QTEXT']


    answers = data['QUESTIONS'][0]['ANSWER'][0]['ATOKEN']
    ans_list = []
    for answer in answers:
        ans_list.append(answer['text'])
        

    line = f'{question}\t{str(ans_list)}'
    print(line,file=fp)

fp.close()



## RESULT MAP CALCULATE

In [15]:
# x , y are list
def cal_ap_score( results , relates):
    ap = 0.0
    idx = 0
    count = 0
    for result in results:
        if result in relates:
            count += 1
            ap += count / (idx + 1)
        idx += 1
#         print(ap)
            
    ap /= len(relates)
    if ap > 1.0:
        print('ap error')
    return ap


In [16]:
import pickle
file_path = './DPR/ctx_embedding_output/result1_1_all_bm25'

with open(file_path) as f:
    results = json.load(f)

print(len(results))

4327


In [17]:
results[0]

{'question': 'what does Grégoire Colin appear in?',
 'answers': ['Before the Rain'],
 'ctxs': [{'id': 'wiki-entities_qa_actor_to_movie_test_1_1',
   'title': '',
   'text': 'Grégoire Colin (born 27 July 1975) is a French actor, producer, screenwriter, and director.Before the Rain (Macedonian: Пред дождот, Pred doždot) is a 1994 film written and directed by Milcho Manchevski.The film stars Katrin Cartlidge, Rade Šerbedžija, Grégoire Colin and Labina Mitevska.It features an original score by the Macedonian band Anastasia.Before the Rain consists of three interlocking stories set both in the Republic of Macedonia and London.The film addresses the ethnic and religious roots of violence and hatred in the Balkans as well as some international implications of the tragic regional conflict.',
   'score': '92.93085',
   'has_answer': True},
  {'id': 'wiki-entities_qa_writer_to_movie_train_327_1',
   'title': '',
   'text': 'Paul Anthelme Bourde (23 May 1851 – 27 October 1914) was a French journa

In [18]:
question2id = {}
for data in test_moviedata:
    did = data['DID']
    question = data['QUESTIONS'][0]['QTEXT']
    if 'Temuera Morrison act' in question:
        print(question)
    question2id[question] = did
    
print(len(question2id))
print(len(test_moviedata))

4327
4327


In [19]:
result_passages = []
for result in results:

        question = result['question']
        did = question2id[question]
        ctxs = result['ctxs']
        label_passages = movie_question2passages[question]

        if len(label_passages) == 0:
            result_passages.append([])
            continue
        predict_passages = []

        for ctx in ctxs:
            ctx_id = ctx['id']
            ctx_text = ctx['text']
            score = ctx['score']
            has_answer = bool(ctx['has_answer'])
            predict_passages.append(ctx_text)
            if len(predict_passages) >= 500:
                break
        result_passages.append(predict_passages)
        
print(len(results))
print(len(result_passages))
                
        

4327
4327


In [20]:
for map_at in [1,3,5,10,50,100,500]:
#     map_at = 1
    map_score = 0.0
    avg_len_of_label = 0 
    count = 0

    for i in range(len(results)):
        result = results[i]

        question = result['question']
        did = question2id[question]
        ctxs = result['ctxs']
        label_passages = movie_question2passages[question]

        if len(label_passages) == 0:
            continue


#         predict_passages = result_passages[i]
        predict_passages = []

        for ctx in ctxs:
            ctx_id = ctx['id']
            ctx_text = ctx['text']
            score = ctx['score']
            has_answer = bool(ctx['has_answer'])
            predict_passages.append(ctx_text)
            if len(predict_passages) >= map_at:
                break
                
        predict_passages = list(OrderedDict(zip(predict_passages, repeat(None))))
        predict_passages = predict_passages[:map_at]
        ap_score = cal_ap_score(predict_passages, label_passages)

        map_score += ap_score
        avg_len_of_label += len(label_passages)
        count += 1

    print('mapat',map_at)  
    print(map_score/count)



mapat 1
0.6036549207622527
mapat 3
0.6458781541803689
mapat 5
0.655799418146472
mapat 10
0.6610406667920501
mapat 50
0.6642555546591238
mapat 100
0.6645997969487774
mapat 500
0.6647703809584437


# BM25 score

In [21]:
# BM25 preprocessing
from nltk.stem.porter import PorterStemmer
import string
import re
from gensim.summarization import bm25

p_stemmer = PorterStemmer()  

processed_all_chunks = []
for chunk in all_chunks:
    chunk = process_chunk(chunk)

    chunk_split = chunk.split()
    stemmed_tokens = [p_stemmer.stem(i) for i in chunk_split]
    processed_all_chunks.append(stemmed_tokens)
    
bm25_obj = bm25.BM25(processed_all_chunks)

In [22]:
print(len(processed_all_chunks))
print(len(all_chunks))

86360
86360


In [23]:
result_bm_passages = []

for i in range(len(test_moviedata)):
    data = test_moviedata[i]
    question = data['QUESTIONS'][0]['QTEXT']
    did = question2id[question]
    label_passages = movie_question2passages[question]
    
    if len(label_passages) == 0:
        result_bm_passages.append([])
        continue

    avg_len_of_label += len(label_passages)
    count += 1
    
    question = process_chunk(question)
    question_split = question.split()
    question_stemmed_tokens = [p_stemmer.stem(i) for i in question_split]

    scores = bm25_obj.get_scores(question_stemmed_tokens)
    topk = 500
    predict_passages = []
    
    scores = np.array(scores)
    best_docs = scores.argsort()[-topk:][::-1]

    for doc_idx in best_docs:
        chunk = all_chunks[doc_idx]
        predict_passages.append(chunk)

    result_bm_passages.append(predict_passages)

print(len(result_bm_passages))
    
    

4327


In [24]:
for map_at in [1,3,5,10,50,100,500]:
    map_score = 0.0
    len_count = 0
    avg_len_of_label = 0 
    count = 0

    for i in range(len(test_moviedata)):
        data = test_moviedata[i]
        question = data['QUESTIONS'][0]['QTEXT']
        did = question2id[question]
        label_passages = movie_question2passages[question]

        if len(label_passages) == 0:
            continue

        count += 1

        predict_passages = result_bm_passages[i]
        predict_passages = list(OrderedDict(zip(predict_passages, repeat(None))))
        predict_passages = predict_passages[:map_at]
#         print(question)
#         print(predict_passages)
#         print('------')
#         print(label_passages)
        
        ap_score = cal_ap_score( predict_passages , label_passages)

        map_score += ap_score
#         break
    
    print(f'map: {map_at}')
    print(map_score/count)
#     break
# fp.close()

map: 1
0.27171450888735027
map: 3
0.35374848682725973
map: 5
0.37246230378020156
map: 10
0.38462457620850915
map: 50
0.392098316464106
map: 100
0.39268528539626074
map: 500
0.39300760526341194


In [46]:
import numpy as np
question = 'what does Grégoire Colin appear in?'
question = process_chunk(question)
question_split = question.split()
question_stemmed_tokens = [p_stemmer.stem(i) for i in question_split]

scores = bm25_obj.get_scores(question_stemmed_tokens)
topk = 3
predict_passages = []


print(len(scores))
scores = np.array(scores)
best_docs = scores.argsort()[-topk:][::-1]

print(best_docs)
for doc_idx in best_docs:
    chunk = all_chunks[doc_idx]
    print(chunk)

86360
[47207 24920 49214]
Grégoire Colin (born 27 July 1975) is a French actor, producer, screenwriter, and director.Before the Rain (Macedonian: Пред дождот, Pred doždot) is a 1994 film written and directed by Milcho Manchevski.The film stars Katrin Cartlidge, Rade Šerbedžija, Grégoire Colin and Labina Mitevska.It features an original score by the Macedonian band Anastasia.Before the Rain consists of three interlocking stories set both in the Republic of Macedonia and London.The film addresses the ethnic and religious roots of violence and hatred in the Balkans as well as some international implications of the tragic regional conflict.
Labina Mitevska (Macedonian: Лабина Митевска) (born 1975 in Skopje, Socialist Republic of Macedonia, Yugoslavia) is a Macedonian actress.Before the Rain (Macedonian: Пред дождот, Pred doždot) is a 1994 film written and directed by Milcho Manchevski.The film stars Katrin Cartlidge, Rade Šerbedžija, Grégoire Colin and Labina Mitevska.It features an origin